In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import re

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

data_train = pd.read_csv('./train.csv', index_col = "id")
data_test = pd.read_csv('./test.csv', index_col = "id")

In [2]:
data_train.head(5)

,text,target
id,,
0,"\nThey were, and even if Washington might cons...",10
1,"We run ""SpaceNews & Views"" on our STAREACH BBS...",14
2,\n\n\nNot to worry. The Masons have been demo...,19
3,"Only Brendan McKay, or maybe ARF, would come t...",17
4,Help: I am running some sample problems from O...,5


In [3]:
data_train.shape

(9233, 2)

In [4]:
data_test.head(5)

,text
id,
0,\nThe VL-IDE Adapter can be much faster then t...
1,\n\nYeah. In a fire that reportedly burned ho...
2,":Judge: ""I grant you immunity from whatever ma..."
3,"I, too, put a corbin seat on my Hawk. I got t..."
4,\n\nDo I ever!!!!!! After 2 years of having h...


In [5]:
data_test.shape

(9233, 1)

In [6]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

In [7]:
data_train.iloc[1]['text']

'We run "SpaceNews & Views" on our STAREACH BBS, a local\noperation running WWIV software with the capability to link to\nover 1500 other BBS\'s in the U.S.A. and Canada through WWIVNet.\n   Having just started this a couple of months ago, our sub us\ncurrently subscribed by only about ten other boards, but more\nare being added.\n   We get our news articles re on Internet, via ftp from NASA\nsites, and from a variety of aerospace related periodicals. We\nget a fair amount of questions on space topics from students\nwho access the system.\n ____________________________________________________________'

In [8]:
preprocess_text(data_train.iloc[1]['text'])

'we run spacenews views on our stareach bbs , a local operation running wwiv software with the capability to link to over other bbs s in the u . s . a . and canada through wwivnet . having just started this a couple of months ago , our sub us currently subscribed by only about ten other boards , but more are being added . we get our news articles re on internet , via ftp from nasa sites , and from a variety of aerospace related periodicals . we get a fair amount of questions on space topics from students who access the system . '

In [9]:
preprocess_list = []
for i in range(data_train.shape[0]):
    preprocess_list.append(preprocess_text(data_train.iloc[i]['text']))
data_train['text'] = preprocess_list

preprocess_list = []
for i in range(data_test.shape[0]):
    preprocess_list.append(preprocess_text(data_test.iloc[i]['text']))
data_test['text'] = preprocess_list

In [10]:
data_train.head(5)

,text,target
id,,
0,"they were , and even if washington might cons...",10
1,"we run spacenews views on our stareach bbs , a...",14
2,not to worry . the masons have been demonized...,19
3,"only brendan mckay , or maybe arf , would come...",17
4,help i am running some sample problems from o ...,5


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
#import tensorflow as tf

vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df = 0.5, use_idf = True)
vectorizer.fit(np.array(data_train["text"]))

train_X = vectorizer.transform(data_train["text"])
train_y = data_train["target"]
X_test = vectorizer.transform(data_test["text"])
'''
train_y = tf.one_hot(train_y, 20)
train_y = np.array(train_y)
train_y = train_y.reshape(-1, 20)
'''

'\ntrain_y = tf.one_hot(train_y, 20)\ntrain_y = np.array(train_y)\ntrain_y = train_y.reshape(-1, 20)\n'

In [12]:
train_X.shape

(9233, 722594)

In [13]:
train_y.shape

(9233,)

In [14]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier

skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True) #총 10번의 fold 진행
n = 0 #x번째 fold인지 기록

fold_target_pred = []

for train_index, valid_index in skf.split(train_X, train_y):
    n += 1
    val_pred_name = [] #validation pred model 이름 저장
    val_pred = []      #validation set pred 결과 저장
    target_pred = []   #test set pred 결과 저장
    
    X_train, X_valid = train_X[train_index], train_X[valid_index]
    y_train, y_valid = train_y[train_index], train_y[valid_index]
    
    model = SGDClassifier(tol=1e-4, power_t=0.3)
    model.fit(X_train, y_train)
    model_pred = model.predict(X_valid)
    fold_target_pred.append(model.predict(X_test))
    print("sgd 모델 정확도 %.4f" % accuracy_score(y_valid, model_pred))
    
    model = LinearSVC()
    model.fit(X_train, y_train)
    model_pred = model.predict(X_valid)
    fold_target_pred.append(model.predict(X_test))
    print("svc 모델 정확도 %.4f" % accuracy_score(y_valid, model_pred))

sgd 모델 정확도 0.7716
svc 모델 정확도 0.7749
sgd 모델 정확도 0.7771
svc 모델 정확도 0.7749
sgd 모델 정확도 0.7478
svc 모델 정확도 0.7413
sgd 모델 정확도 0.7497
svc 모델 정확도 0.7497
sgd 모델 정확도 0.7746
svc 모델 정확도 0.7790
sgd 모델 정확도 0.7389
svc 모델 정확도 0.7400
sgd 모델 정확도 0.7194
svc 모델 정확도 0.7205
sgd 모델 정확도 0.7497
svc 모델 정확도 0.7551
sgd 모델 정확도 0.7400
svc 모델 정확도 0.7432
sgd 모델 정확도 0.7281
svc 모델 정확도 0.7259


In [15]:
np.array(fold_target_pred).shape

(20, 9233)

In [16]:
from collections import Counter

final_pred = []
for i in range(9233):
    pred = []
    for x in range(20):
        pred.append(fold_target_pred[x][i])
    count_pred = Counter(pred)
    max_item = count_pred.most_common(n=1)
    final_pred.append(max_item[0][0])

In [17]:
final_pred

[3,
 16,
 11,
 8,
 13,
 4,
 8,
 13,
 12,
 3,
 8,
 5,
 14,
 10,
 12,
 7,
 18,
 19,
 11,
 8,
 7,
 14,
 14,
 5,
 1,
 10,
 0,
 13,
 10,
 9,
 17,
 5,
 11,
 15,
 17,
 7,
 14,
 6,
 12,
 3,
 2,
 9,
 6,
 13,
 3,
 16,
 17,
 3,
 5,
 17,
 11,
 17,
 0,
 8,
 2,
 2,
 15,
 10,
 1,
 10,
 11,
 3,
 2,
 9,
 12,
 10,
 14,
 17,
 6,
 15,
 18,
 10,
 15,
 17,
 3,
 3,
 12,
 17,
 1,
 13,
 8,
 1,
 10,
 14,
 9,
 15,
 16,
 3,
 12,
 11,
 16,
 16,
 8,
 6,
 8,
 4,
 10,
 3,
 16,
 0,
 4,
 2,
 6,
 10,
 5,
 3,
 3,
 5,
 15,
 6,
 3,
 6,
 2,
 1,
 4,
 11,
 18,
 5,
 8,
 8,
 7,
 12,
 2,
 2,
 0,
 18,
 19,
 7,
 6,
 0,
 13,
 8,
 13,
 2,
 6,
 5,
 12,
 13,
 5,
 0,
 7,
 10,
 0,
 6,
 14,
 17,
 16,
 8,
 13,
 14,
 10,
 16,
 11,
 4,
 8,
 5,
 10,
 11,
 5,
 9,
 11,
 16,
 14,
 2,
 16,
 17,
 0,
 15,
 10,
 3,
 17,
 14,
 11,
 3,
 13,
 7,
 11,
 17,
 16,
 7,
 4,
 13,
 15,
 3,
 7,
 17,
 16,
 4,
 3,
 2,
 1,
 2,
 9,
 6,
 3,
 1,
 6,
 13,
 7,
 4,
 14,
 16,
 6,
 13,
 12,
 6,
 17,
 3,
 12,
 14,
 5,
 0,
 13,
 14,
 14,
 12,
 16,
 13,
 0,
 5,
 3,
 6,
 16,

In [18]:
submission = pd.read_csv("./sample_submission.csv")
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [19]:
submission['target'] = final_pred
submission.head()

,id,target
0,0,3
1,1,16
2,2,11
3,3,8
4,4,13


In [20]:
submission.to_csv("Submit_2.csv",index=False)